# Librerías

Importamos las librerías necesarias y cargamos variables de entorno

In [16]:
%pip install redis
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from apis.redis_api import RedisAPI
from apis.messageData import MessageData
from apis.relationData import make_rel_id
from dotenv import load_dotenv
import os

load_dotenv(ROOT / ".env.local")

REDIS_HOST = os.getenv("REDIS_HOST")
REDIS_PORT = os.getenv("REDIS_PORT")
REDIS_DB = os.getenv("REDIS_DB")

api = RedisAPI(REDIS_HOST, REDIS_PORT, REDIS_DB)
# Reset inicial
api.clear_all_queues()

9

# Operaciones de colas

# Crear Cola

Una cola se crea al encolar un mensaje, no hay metodo "create_queue()"

enqueue_message() inserta un mensaje al final de la cola (RPUSH). Devuelve el número de elementos

In [3]:
message = MessageData(
    sender_id="u1",
    receiver_id="u2",
    content="Hola u2, este es un mensaje de u1.",
    rel_id=make_rel_id("u1", "u2")
)
for _ in range(3):
    api.enqueue_message(message.rel_id, message.to_redis_dict())

In [4]:
message2 = MessageData(
    sender_id="u2",
    receiver_id="u1",
    content="Hola u1, este es un mensaje de u2.",
    rel_id=make_rel_id("u2", "u1")
)
for _ in range(2):
    api.enqueue_message(message2.rel_id, message2.to_redis_dict())

# Mostrar mensajes de una cola sin consumir

peek_messages(rel_id, start, end) muestra los mensajes de una relación. Se puede definir inicio y fin de los mensajes (Muestra todo sino)

In [5]:
messages = api.peek_messages(make_rel_id("u1", "u2"))
for msg in messages:
    print(msg)

{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}
{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}
{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}


# Mostrar Colas

list_queues() muestra todas las colas actuales

In [6]:
api.list_queues()

['queue:u2-u1', 'queue:u1-u2']

# Longitud de la cola

queue_length(rel_id) indica la longitud de la cola (Se puede interpretar como número de mensajes)

In [7]:
numMessages = api.queue_length(make_rel_id("u1", "u2"))
print(f"Número de mensajes en la cola u1-u2: {numMessages}")

Número de mensajes en la cola u1-u2: 3


# Actualizar mensaje
update_message_at(rel_id, index, new_message) actualiza un mensaje dentro de una cola en una posición específica. Devuelve True si lo logró, False en caso contrario

In [8]:
new_message = MessageData(
    sender_id="u1",
    receiver_id="u2",
    content="Adios u2, este es un mensaje de u1.",
    rel_id=make_rel_id("u1", "u2")
)
api.update_message_at(new_message.rel_id, 0, new_message.to_redis_dict())

True

In [9]:
messages = api.peek_messages(make_rel_id("u1", "u2"))
for msg in messages:
    print(msg)

{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Adios u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '93e37c3e-ef6f-4153-ba10-b5531a914150', 'timestamp': '2026-01-11T17:47:22.661355+00:00', 'metadata': {}}
{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}
{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}


# Borrar mensaje

delete_message_at(rel_id, index) elimina un mensaje de una cola en una posición específica. Devuelve True si lo logró, False si no

In [10]:
api.delete_message_at(make_rel_id("u2", "u1"), 1)

True

In [11]:
messages = api.peek_messages(make_rel_id("u2", "u1"))
if not messages:
    print("No hay mensajes en la cola u2-u1")
else:
    for msg in messages:
        print(msg)

{'sender_id': 'u2', 'receiver_id': 'u1', 'content': 'Hola u1, este es un mensaje de u2.', 'rel_id': 'u2-u1', 'msg_id': '352e2af0-44e3-4bb8-afeb-19882311c801', 'timestamp': '2026-01-11T17:47:22.624018+00:00', 'metadata': {}}


# Consumir mensaje
consume_blocking(rel_id, timeout) consume el primer mensaje de la cola. Si no hay mensajes y se supera el timeout, devuelve None (BLPOP)

In [12]:
api.consume_blocking(make_rel_id("u1", "u2"))

MessageData(sender_id='u1', receiver_id='u2', content='Adios u2, este es un mensaje de u1.', rel_id='u1-u2', msg_id='93e37c3e-ef6f-4153-ba10-b5531a914150', timestamp=datetime.datetime(2026, 1, 11, 17, 47, 22, 661355, tzinfo=datetime.timezone.utc), metadata={})

In [13]:
messages = api.peek_messages(make_rel_id("u1", "u2"))
if not messages:
    print("No hay mensajes en la cola u1-u2")
else:
    for msg in messages:
        print(msg)

{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}
{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}


# Exportar e Importar colas

# Exportar colas
export_queues_state() exporta todas las colas actuales

In [14]:
queues = api.export_queues_state()
for queue in queues:
    print(f"Cola: {queue}")

Cola: {'key': 'queue:u2-u1', 'rel_id': 'u2-u1', 'items': [{'sender_id': 'u2', 'receiver_id': 'u1', 'content': 'Hola u1, este es un mensaje de u2.', 'rel_id': 'u2-u1', 'msg_id': '352e2af0-44e3-4bb8-afeb-19882311c801', 'timestamp': '2026-01-11T17:47:22.624018+00:00', 'metadata': {}}]}
Cola: {'key': 'queue:u1-u2', 'rel_id': 'u1-u2', 'items': [{'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}, {'sender_id': 'u1', 'receiver_id': 'u2', 'content': 'Hola u2, este es un mensaje de u1.', 'rel_id': 'u1-u2', 'msg_id': '0c758ffe-1c08-4148-91c9-b6204841fbc3', 'timestamp': '2026-01-11T17:47:22.614173+00:00', 'metadata': {}}]}


# Importar colas
import_queues_state(queues) importa las colas recibidas por parametros

In [15]:
api.clear_all_queues()
api.import_queues_state(queues)
queues = api.list_queues()
print(f"Colas después de la importación: {queues}")

Colas después de la importación: ['queue:u2-u1', 'queue:u1-u2']
